<a href="https://colab.research.google.com/github/flaviopradoaquino/TCC_MBA_USP_ESALQ/blob/main/TCC_USP_FaceBookPro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação e Importação das Bibliotecas**

In [ ]:
!pip install prophet
!pip install -q yfinance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import yfinance as yf
import datetime
import plotly.express as px
from prophet.plot import plot_plotly, plot_components_plotly
from datetime import date, timedelta
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error

# **Construção do Dataset**

In [ ]:
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')

In [ ]:
from datetime import datetime 
dt_atual = date.today()
td = timedelta(1095)
dt_final = datetime(2023, 3, 3)

In [ ]:
### Data de referência menos 1095 dias
df = yf.download("BOVA11.SA", start=(dt_final-td))['Close']

### Data atual menos 1095 dias
# df = yf.download("BOVA11.SA", start=(dt_atual-td))['Close']

### Intervalo de datas fixas
# df = yf.download("BOVA11.SA", start="2020-03-03", end="2023-03-03")['Close']

In [ ]:
df

In [ ]:
df = df.reset_index()
df

In [ ]:
df.info()

In [ ]:
df_bova_temp = df[['Date', 'Close']].rename(columns = {'Date': 'ds', 'Close': 'y'})
df_bova_temp

In [ ]:
### Alocando somente a data e excluindo uma possível hora inclusa na importação dos dados
df_bova = pd.DataFrame({
  "ds": df_bova_temp['ds'].dt.date,
  "y":df_bova_temp['y']
})

# **Excluindo Outliers**

In [ ]:
dados = df_bova['y']
dados.describe()

In [ ]:
boxplot_bova = px.box(dados, y="y")
boxplot_bova.show()

In [ ]:
q1 = np.quantile(dados, .25, method="midpoint")
q2 = np.quantile(dados, .5, method="midpoint")
q3 = np.quantile(dados, .75, method="midpoint")
IIQ = q3 - q1

In [ ]:
### Limite inferior do boxplot
limite_inferior = q1 - 1.5 * IIQ
limite_inferior

In [ ]:
### Limite superior do boxplot
limite_superior = q3 + 1.5 * IIQ
limite_superior

In [ ]:
df_bova_lim_inf = df_bova[df_bova['y']>limite_inferior]
df_bova_clean = df_bova_lim_inf[df_bova_lim_inf['y']<limite_superior]
df_bova = df_bova_clean
df_bova


# **Construção do Modelo**

In [ ]:
### Hperparâmetros do modelo
md = Prophet(
             n_changepoints=20, 
             seasonality_mode='additive', 
             seasonality_prior_scale=10, 
             weekly_seasonality=False,
             daily_seasonality=False,
             interval_width=0.95
             )
### Adicionando feriados do Brasil
md.add_country_holidays(country_name='BR')
md.fit(df_bova)

In [ ]:
tempo = md.make_future_dataframe(periods=90)
previsoes = md.predict(tempo)

In [ ]:
previsoes.head()

In [ ]:
previsoes.head(3).T

In [ ]:
### Conferindo a aderência dos feriados ao modelo
md.train_holiday_names

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

In [ ]:
plot_plotly(md, previsoes)

In [ ]:
plot_components_plotly(md, previsoes)

# **Avaliação do Modelo**

In [ ]:
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')
### Definição do intervalo de tempo
dt_atual = date.today()
td = timedelta(1095)

In [ ]:
### Importação dos dados do website da Yahoo Finance
df2 = yf.download("BOVA11.SA", start=(dt_atual-td))['Close']

In [ ]:
### Contando o número de observações gerado
len(df2.index)

In [ ]:
### Definindo o dataset de avaliação a partir da linha 365
df_aval = df2[len(df2.index)-365:]
df_aval.shape

In [ ]:
df2

In [ ]:
### Gerando o boxblot pela variável com o valor de fechamento pregão do dia (Close)
boxplot_avaliacao = px.box(df_aval, y="Close")
boxplot_avaliacao.show()

In [ ]:
df2.shape

In [ ]:
len(df2.index)-365

In [ ]:
teste = df2[len(df2.index)-365:]
teste.shape

In [ ]:
teste.head()

In [ ]:
pred = md.make_future_dataframe(periods=0)
previsao = md.predict(pred)

In [ ]:
previsao = previsao['yhat'].tail(365)

In [ ]:
previsao

In [ ]:
teste = teste.reset_index()
teste

In [ ]:
teste = teste[['Close']]
teste

In [ ]:
previsao = previsao.reset_index()
previsao = previsao[['yhat']]
previsao

In [ ]:
mean_absolute_error(teste, previsao)